# Going Live with Algo Trading

In [181]:
# Initial imports
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

# Load .env environment variables
load_dotenv()

True

## Alpaca Example
**Note**: You'll need to use your own API and Secret key here to run this code. (You can do so for free at the [Alpaca Trading](https://alpaca.markets/) website).

[alpaca-trade-api ](https://github.com/alpacahq/alpaca-trade-api-python) is a python package that can be used to trade algorithmically through [Alpaca](https://alpaca.markets/), a U.S. equities broker.

In [182]:
import alpaca_trade_api as tradeapi

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

In [183]:
# Create a connection to the API 
api = tradeapi.REST(API_KEY, API_SECRET, ALPACA_API_BASE_URL, api_version="v2")

In [184]:
# Set the ticket symbol and the number of shares to buy
ticker = "TSLA"
number_of_shares = 1

In [185]:
# Make API call
prices = api.get_bars(ticker, "1Min").df

# Reorganize the DataFrame
prices = pd.concat([prices], axis=1)

# Get final closing price
limit_amount = prices["close"][-1]

prices = prices.reset_index()

prices = prices.rename(columns = {'timestamp' : 'date'})
prices.drop(prices.columns[[6, 7]], axis=1, inplace=True) 

prices['date'] = pd.to_datetime(
    prices['date']
)

prices = prices.set_index('date')

prices

,open,high,low,close,volume
date,,,,,
2022-10-19 08:00:00+00:00,222.09,224.13,220.76,220.76,25316
2022-10-19 08:01:00+00:00,220.82,221.23,220.80,221.00,15679
2022-10-19 08:02:00+00:00,220.90,221.20,220.90,220.90,15018
2022-10-19 08:03:00+00:00,220.94,221.10,220.82,220.97,16317
2022-10-19 08:04:00+00:00,220.98,221.13,220.91,221.12,15331
...,...,...,...,...,...
2022-10-19 23:37:00+00:00,210.18,210.25,210.10,210.10,9886
2022-10-19 23:38:00+00:00,210.10,210.20,210.00,210.10,33890
2022-10-19 23:39:00+00:00,210.05,210.15,210.00,210.00,19267


In [186]:
from finta import TA
import numpy as np

short_window = 7
long_window = 100 

prices['Short'] = TA.SMA(prices, short_window)
prices['Long'] = TA.SMA(prices, long_window)

# Set the Signal column
prices["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where 0 is when the Short window is under the Long window
prices["Signal"][short_window:] = np.where(
    prices["Short"][short_window:] > prices["Long"][short_window:], 1.0, 0.0
)

# Review the DataFrame
prices

c:\Users\minha\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,open,high,low,close,volume,Short,Long,Signal
date,,,,,,,,
2022-10-19 08:00:00+00:00,222.09,224.13,220.76,220.76,25316,NaN,NaN,0.0
2022-10-19 08:01:00+00:00,220.82,221.23,220.80,221.00,15679,NaN,NaN,0.0
2022-10-19 08:02:00+00:00,220.90,221.20,220.90,220.90,15018,NaN,NaN,0.0
2022-10-19 08:03:00+00:00,220.94,221.10,220.82,220.97,16317,NaN,NaN,0.0
2022-10-19 08:04:00+00:00,220.98,221.13,220.91,221.12,15331,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
2022-10-19 23:37:00+00:00,210.18,210.25,210.10,210.10,9886,210.207143,211.033997,0.0
2022-10-19 23:38:00+00:00,210.10,210.20,210.00,210.10,33890,210.188571,211.007997,0.0
2022-10-19 23:39:00+00:00,210.05,210.15,210.00,210.00,19267,210.145714,210.984697,0.0


In [187]:
import hvplot.pandas 

# Visualize the close price for the investment
security_close = prices[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize the moving averages
moving_avgs = prices[['Short', 'Long']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay the plots
entry_exit_plot = security_close * moving_avgs
entry_exit_plot.opts(
    title="TSLA -- Dual Moving Average Trading Algorithm"
)

:Overlay
   .Curve.I     :Curve   [date]   (close)
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [date]   (value)

In [ ]:
# Create buy signal, num shares and ticker
if (prices['Signal'] == 1).any():
    orderSide = "buy"
if (prices['Signal'] == 0).any():
    orderSide = "sell"

In [ ]:
# Submit order
api.submit_order(
    symbol="TSLA", 
    qty=number_of_shares, 
    side=orderSide, 
    time_in_force="day", 
    type="limit", 
    limit_price=limit_amount,
    extended_hours = True
)